In [236]:
import random
import copy

import pandas as pd
import numpy as np

import wandb

from darts import TimeSeries
from darts.metrics.metrics import mae, rmse, mape, smape
from darts.models import BlockRNNModel

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader
from torcheval.metrics.functional import binary_accuracy, binary_f1_score, binary_precision, binary_recall

In [237]:
def evaluate_regression_metrics(actual, pred):
    actual = actual.squeeze()
    actual = TimeSeries.from_series(pd.DataFrame(actual))
    pred = TimeSeries.from_series(pd.DataFrame(pred).squeeze())
    result = {}
    result["MAE"] = mae(actual, pred)
    result["RMSE"] = rmse(actual, pred)
    result["MAPE"] = mape(actual, pred)
    result["SMAPE"] = smape(actual, pred)
    return result

def evaluate_classification_metrics(actual, pred):
    actual = actual.squeeze()
    pred = pred.squeeze()
    
    result = {}
    result["Accuracy"] = binary_accuracy(actual, pred)
    result["F1"] = binary_f1_score(actual, pred)
    result["Precision"] = binary_precision(actual, pred)
    result["Recall"] = binary_recall(actual, pred)
    return result

In [238]:
def binarize_values(X: torch.Tensor, values: torch.Tensor) -> torch.Tensor:
    """
    Converts continuous predictions into binary classification labels.

    Args:
    - X (torch.Tensor): Tensor of past values (last observed values of sequences).
    - values (torch.Tensor): Tensor of target or predicted values.

    Returns:
    - torch.Tensor: Binary tensor (1 if increase, 0 otherwise).
    """
    return (values > X).int()


In [239]:
def evaluate_test(model, test_loader, device, loss_fn):
    model.eval()
    outputs_list = []
    labels_list = []
    past_values_list = []
    test_loss = 0

    with torch.no_grad():
        inputs, labels = next(iter(test_loader))
        inputs, labels = inputs.to(device), labels.to(device).float().squeeze()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        test_loss += loss.item()
        
        outputs_list.append(outputs.cpu())
        labels_list.append(labels.cpu())
        past_values_list.append(inputs[:,0].cpu())

    outputs_concat = torch.cat(outputs_list, dim=0)
    labels_concat = torch.cat(labels_list, dim=0)
    past_values_concat = torch.cat(past_values_list, dim=0)

    regression_metrics = evaluate_regression_metrics(labels_concat, outputs_concat)
    actual_direction = binarize_values(past_values_concat, labels_concat)
    preds_direction = binarize_values(past_values_concat, outputs_concat)
    classification_metrics = evaluate_classification_metrics(actual_direction, preds_direction)

    print(f'Test Loss: {test_loss}')
    print(f'Regression Metrics: {regression_metrics}')
    print(f'Classification Metrics: {classification_metrics}')
    
    wandb.log({
        'test_loss': test_loss,
        **regression_metrics,
        **classification_metrics
    })
    
    return test_loss, regression_metrics, classification_metrics

In [240]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, n_past, n_predict, n_stride=1):
        """
        Args:
            data: Input time series data.
            n_past: Number of past time steps as input.
            n_predict: Number of future time steps to predict.
            n_stride: Step size for moving window.
        """
        self.data = data
        self.n_past = n_past
        self.n_predict = n_predict
        self.n_stride = n_stride
        self.samples = self._create_samples()
    
    def _create_samples(self):
        samples = []
        for i in range(0, len(self.data) - self.n_past - self.n_predict + 1, self.n_stride):
            past = self.data[i : i + self.n_past]
            future = self.data[i + self.n_past : i + self.n_past + self.n_predict]
            samples.append((past, future))  # Directly use the future values as targets
        return samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        past, target = self.samples[idx]
        return (torch.tensor(past.squeeze(-1), dtype=torch.float32), 
                torch.tensor(target, dtype=torch.float32))  # Always return continuous values for regression


In [241]:
class Trainer:
    def __init__(self, model, device, optimizer, loss_fn, train_loader, val_loader, n_epochs,
                 run_name, patience=50):
        self.device = device
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.n_epochs = n_epochs
        self.patience = patience
        self.best_val_loss = float('inf')
        self.patience_counter = 0
        self.best_model_state = None
        self.early_stopped = False
        self.run_name = run_name
        
        
    def train(self):
        for epoch in range(self.n_epochs):
            train_loss = self._train_epoch(epoch)
            val_loss, regression_metrics, classification_metrics= self._val_epoch(epoch)
            wandb.log({"train_loss": train_loss, "val_loss": val_loss})
            wandb.log({
            **regression_metrics,
            **(classification_metrics if classification_metrics else {})
        })
            self._early_stopping(val_loss)
            if self.early_stopped:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break
            
        print("Training complete, saving best and final model.")
        self._save_best_model()
        self._save_final_model()
        
        best_model = copy.deepcopy(self.model)  
        best_model.load_state_dict(self.best_model_state)
        return best_model 
            
    def _train_epoch(self, epoch):
        self.model.train()
        train_loss = 0
        for batch in self.train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.loss_fn(outputs, labels.float().squeeze())
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()
        print(f'Epoch {epoch+1}, Train Loss: {train_loss / len(self.train_loader)}')
        return train_loss / len(self.train_loader)
    
    def _val_epoch(self, epoch):
        self.model.eval()
        val_loss = 0
        outputs_list = []
        labels_list = []
        past_values_list = []
        
        with torch.no_grad():
            for batch in self.val_loader:
                inputs, labels = batch
                inputs, labels = inputs.to(self.device), labels.to(self.device).float().squeeze()
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, labels)
                val_loss += loss.item()
                outputs_list.append(outputs.cpu())
                labels_list.append(labels.cpu())
                past_values_list.append(inputs[:, -1].cpu()) 
                
        outputs_concat = torch.cat(outputs_list, dim=0)
        labels_concat = torch.cat(labels_list, dim=0)
        past_values_concat = torch.cat(past_values_list, dim=0)
        
        
        regression_metrics = evaluate_regression_metrics(labels_concat, outputs_concat)
        actual_direction = binarize_values(past_values_concat, labels_concat)
        preds_direction = binarize_values(past_values_concat, outputs_concat)
        classification_metrics = evaluate_classification_metrics(actual_direction, preds_direction)
        
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss / len(self.val_loader)}')
        print(f"regression_metrics", regression_metrics)
        print(f"Classification Metrics: {classification_metrics}")
        return val_loss / len(self.val_loader), regression_metrics, classification_metrics
    
    def _early_stopping(self, val_loss):
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.patience_counter = 0
            self.best_model_state = self.model.state_dict()
        else:
            self.patience_counter += 1
            if self.patience_counter >= self.patience:
                self.early_stopped = True
                
                
    def _save_best_model(self):
        if self.best_model_state is not None:
            torch.save(self.best_model_state, f"../artifacts/{self.run_name}_best_model.pth")
            artifact = wandb.Artifact(f"{self.run_name}_best_model", type="model")
            artifact.add_file(f"../artifacts/{self.run_name}_best_model.pth")
            wandb.log_artifact(artifact)
            print("Best model saved to WandB.")

    def _save_final_model(self):
        torch.save(self.model.state_dict(), f"../artifacts/{self.run_name}_final_model.pth")
        artifact = wandb.Artifact(f"{self.run_name}_final_model", type="model")
        artifact.add_file(f"../artifacts/{self.run_name}_final_model.pth")
        wandb.log_artifact(artifact)
        print("Final model saved to WandB.")

In [242]:
class LinearLayer(nn.Module):
    def __init__(self, n_past, n_predict):
        super().__init__()
        self.fc = nn.Linear(n_past, n_predict)

    def forward(self, x):
        return self.fc(x)

In [243]:
class RNNEncoder(nn.Module):
    def __init__(self, n_predict, n_features, n_hidden, n_layers, dropout):
        super().__init__()
        self.rnn = nn.RNN(input_size = n_features,
                          hidden_size = n_hidden,
                          num_layers = n_layers,
                          nonlinearity = "relu",
                          dropout = dropout,
                          bidirectional = False,
                          batch_first=True)
        self.fc = nn.Linear(n_hidden, n_predict)
        
    def forward(self, x):
        x = x.unsqueeze(-1) 
        output, hn = self.rnn(x)
        x = self.fc(hn[-1])
        return x.squeeze() 

In [244]:
def get_model_class(model_name):
    model_dict = {
        "linear": LinearLayer,
        "rnn_encoder_regression": RNNEncoder
    }
    
    if model_name not in model_dict:
        raise ValueError(f"Unknown model: {model_name}")
    
    return model_dict[model_name] 

In [245]:
def main(config):
    wandb.init(dir="../wandb", project="stock_time_series", name=config["run_name"], config=config, notes=config["notes"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"using {device}")

    
    
    df_train = pd.read_csv("../data/train.csv", parse_dates=["timestamp"])["low"]
    df_val = pd.read_csv("../data/validate.csv", parse_dates=["timestamp"])["low"]
    df_test = pd.read_csv("../data/test.csv", parse_dates=["timestamp"])["low"]
    
    scaler_transformer = None
    if config["scaler"] == "min-max":
        scaler_transformer = MinMaxScaler()
    elif config["scaler"] == "std":
        scaler_transformer = StandardScaler()
        
    train_series = scaler_transformer.fit_transform(df_train.to_frame())
    val_series = scaler_transformer.transform(df_val.to_frame())
    test_series = scaler_transformer.transform(df_test.to_frame())
    
    train_dataset  = TimeSeriesDataset(train_series, 
                                       n_past=config["n_past"], 
                                       n_predict=config["n_predict"], 
                                       n_stride=config["n_stride"])
    val_dataset  = TimeSeriesDataset(val_series, 
                                   n_past=config["n_past"], 
                                   n_predict=config["n_predict"], 
                                   n_stride=config["n_stride"])
    test_dataset  = TimeSeriesDataset(test_series, 
                                   n_past=config["n_past"], 
                                   n_predict=config["n_predict"], 
                                   n_stride=config["n_stride"])
    
    
    train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=False)
    val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)


    model_class = get_model_class(config["model"])
    model = model_class(n_features = config["n_features"], 
                        n_predict = config["n_predict"],
                        n_hidden = config["n_hidden"], 
                        n_layers = config["n_layers"], 
                        dropout = config["dropout"])



    loss_fn = nn.MSELoss()
        
    optimizer = optim.Adam(model.parameters())

    trainer = Trainer(model = model, 
                      device = device, 
                      optimizer = optimizer,
                      loss_fn = loss_fn,
                      train_loader = train_dataloader,
                      val_loader = val_dataloader,
                      n_epochs = config["n_epochs"],
                      run_name = config["run_name"],
                      patience = config["patience"])
    trainer.train()

    evaluate_test(model, test_dataloader, device, loss_fn)
    wandb.finish()
    

In [246]:
def get_config():
    return {
        "run_name" : "rnn_encoder",
        "notes" : "Real Run",  

        "n_features" : 1,
        "n_past" :24,
        "n_predict" : 1,
        "n_stride" : 1,
        
        "model" : "rnn_encoder_regression",
        "n_hidden" : 64,
        "n_layers" : 5,
        "dropout" : 0,
        
        "n_epochs" : 200,
        "batch_size" : 64,
        "patience" :  50,
        "scaler" : "min-max",

    }

In [247]:
if __name__ == "__main__":
    seed = 42
    random.seed(42)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    
    config = get_config()
    main(config) 

using cuda
Epoch 1, Train Loss: 0.00690652460192273
Epoch 1, Validation Loss: 0.14669836959688598
regression_metrics {'MAE': 0.30685827, 'RMSE': 0.3819966, 'MAPE': 25.440838, 'SMAPE': 31.173304}
Classification Metrics: {'Accuracy': tensor(0.4911), 'F1': tensor(0.0617), 'Precision': tensor(0.0327), 'Recall': tensor(0.5394)}
Epoch 2, Train Loss: 0.012944348837717624
Epoch 2, Validation Loss: 0.16104181857455663
regression_metrics {'MAE': 0.3288999, 'RMSE': 0.40026838, 'MAPE': 27.609226, 'SMAPE': 34.030952}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.0030), 'Precision': tensor(0.0015), 'Recall': tensor(0.5000)}
Epoch 3, Train Loss: 0.01504616233494547


Epoch 3, Validation Loss: 0.17499017725139474
regression_metrics {'MAE': 0.3493277, 'RMSE': 0.41727337, 'MAPE': 29.632126, 'SMAPE': 36.746365}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Epoch 4, Train Loss: 0.01379345882625466
Epoch 4, Validation Loss: 0.1543613288895209
regression_metrics {'MAE': 0.31837162, 'RMSE': 0.39186203, 'MAPE': 26.558699, 'SMAPE': 32.65418}
Classification Metrics: {'Accuracy': tensor(0.4888), 'F1': tensor(0.0134), 'Precision': tensor(0.0068), 'Recall': tensor(0.5094)}
Epoch 5, Train Loss: 0.01373676607432248
Epoch 5, Validation Loss: 0.15142935108994354
regression_metrics {'MAE': 0.31427455, 'RMSE': 0.3881186, 'MAPE': 26.16858, 'SMAPE': 32.126724}
Classification Metrics: {'Accuracy': tensor(0.4905), 'F1': tensor(0.0284), 'Precision': tensor(0.0146), 'Recall': tensor(0.5686)}
Epoch 6, Train Loss: 0.01502249025672905
Epoch 6, Validation Loss: 0.15970918569488635
regression_metrics {'MAE':

Epoch 8, Validation Loss: 0.173084813523675
regression_metrics {'MAE': 0.34791052, 'RMSE': 0.41499797, 'MAPE': 29.535149, 'SMAPE': 36.566727}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Epoch 9, Train Loss: 0.016602919717837554
Epoch 9, Validation Loss: 0.16123271021981472
regression_metrics {'MAE': 0.33018488, 'RMSE': 0.4005099, 'MAPE': 27.768171, 'SMAPE': 34.207027}
Classification Metrics: {'Accuracy': tensor(0.4888), 'F1': tensor(0.0005), 'Precision': tensor(0.0003), 'Recall': tensor(1.)}
Epoch 10, Train Loss: 0.018541287367999346


Epoch 10, Validation Loss: 0.1704180899980768
regression_metrics {'MAE': 0.3446668, 'RMSE': 0.4117859, 'MAPE': 29.233486, 'SMAPE': 36.13678}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Epoch 11, Train Loss: 0.019526981883634865


Epoch 11, Validation Loss: 0.165943018425319
regression_metrics {'MAE': 0.33668637, 'RMSE': 0.4063299, 'MAPE': 28.39419, 'SMAPE': 35.062153}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Epoch 12, Train Loss: 0.022748175158875308
Epoch 12, Validation Loss: 0.13996133323771723
regression_metrics {'MAE': 0.29678366, 'RMSE': 0.37310827, 'MAPE': 24.478672, 'SMAPE': 29.895842}
Classification Metrics: {'Accuracy': tensor(0.4915), 'F1': tensor(0.0997), 'Precision': tensor(0.0551), 'Recall': tensor(0.5264)}
Epoch 13, Train Loss: 0.023912640959177636
Epoch 13, Validation Loss: 0.15716716149418958
regression_metrics {'MAE': 0.32295918, 'RMSE': 0.39541543, 'MAPE': 27.019886, 'SMAPE': 33.252846}
Classification Metrics: {'Accuracy': tensor(0.4880), 'F1': tensor(0.0055), 'Precision': tensor(0.0028), 'Recall': tensor(0.4074)}
Epoch 14, Train Loss: 0.022133960199949825


...

Epoch 135, Validation Loss: 0.193923561719292
regression_metrics {'MAE': 0.3753506, 'RMSE': 0.43930846, 'MAPE': 32.211136, 'SMAPE': 40.300224}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Epoch 136, Train Loss: 0.0733109620558344


Epoch 136, Validation Loss: 0.1939237410027045
regression_metrics {'MAE': 0.37535083, 'RMSE': 0.43930864, 'MAPE': 32.21116, 'SMAPE': 40.300255}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Epoch 137, Train Loss: 0.07331097903086854


Epoch 137, Validation Loss: 0.19392387512460596
regression_metrics {'MAE': 0.375351, 'RMSE': 0.43930882, 'MAPE': 32.211178, 'SMAPE': 40.30028}
Classification Metrics: {'Accuracy': tensor(0.4887), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}
Early stopping triggered at epoch 137
Training complete, saving best and final model.
Best model saved to WandB.
Final model saved to WandB.


Test Loss: 0.49862462282180786
Regression Metrics: {'MAE': 0.6908116, 'RMSE': 0.70613354, 'MAPE': 49.663025, 'SMAPE': 66.43101}
Classification Metrics: {'Accuracy': tensor(0.4730), 'F1': tensor(0.), 'Precision': tensor(0.), 'Recall': tensor(0.)}


Accuracy,▂▂▂█▆▂▇▅▃▂█▅▂▄▂▄█▂▄▂▃▂▄▆▃▃▁▁▁▂▆▅▁▂▂▂▂▂▂▂
F1,▂▁▁▁▄▁▁▆█▇▂▅▁▂█▄▁▄▂▁▁▂▂▁▁▁▁▁▁▂▃▆▄▂▁▁▁▁▁▁
MAE,▅▅▆▆▇▃▂▅▄▃▄▄▄▂▄▄▃▃▅▃▄▅▁▁▅▄▅▄▄▅▅█████████
MAPE,▄▄▆▅▆▆▆▄▂▂▄▄▃▆▁▄▂▄▃▅▃▄▃▅▅▅▁▄▁▄▄▂▅▆▆█████
Precision,▁▁▁▁▁▅▆▆▄▃▂▄▁█▅▂▁▂▁▄▂▁▁▁▁▁▁▁▅▂▁▁▁▁▁▁▁▁▁▁
RMSE,▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃█
Recall,▇▁▇▁▇▇▇▅▇▇▆▇▇▆▇▇▁▇▇▇▇▇▁▇▇▆▆██▆▇██▆▇▁▁▁▁▁
SMAPE,▂▂▂▃▂▁▁▂▂▂▂▁▂▃▁▂▃▁▁▂▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃█
test_loss,▁
train_loss,▁▂▂▃▃▃▂▂▃▂▂▂▂▂▃▂▃▅▄▃▆▇▅▂▄▅▅▆▄▃▂▆▇▇██████
val_loss,▅▆▄▃▃▄▃▂▅▃▅▃▄▄▂▄▃▃▄▅▅▄▅▅▅▁▅▅▅▄▄▄▄▅▆█████
